In [40]:
import os
import sys
sys.path.append("C:/Users/yuuta/Documents/fashion")

In [41]:
import glob
import random
from utils.util import is_target_category, open_json
from utils.util import get_progress_percent

random.seed(42)

In [42]:
dirpath = glob.glob('C:/Users/yuuta/Documents/fashion/data/train/**/*.json')
random.shuffle(dirpath)

In [43]:
import importlib
import utils.util
import pandas as pd
importlib.reload(utils.util)

<module 'utils.util' from 'C:\\Users/yuuta/Documents/fashion\\utils\\util.py'>

In [44]:
print(len(dirpath))
random.shuffle(dirpath)

162975


In [45]:
# N-pair sampling
# triplet loss
# csvには，(anchor, positive, negative)で保存 (itemIdごとに)
# dirpath = dirpath[:1000]
N = 128
data = []

In [46]:
for i in range(0, len(dirpath), N):
    print(get_progress_percent(i, len(dirpath)), end='')
    category_items = {}
    
    for coordinate_path in dirpath[i:i+N]:
        coordinate = open_json(coordinate_path)
        # targetのもののみに絞る
        items = list(filter(is_target_category, coordinate["items"]))

        for item in items:
            category = item["category x color"]
            if category not in category_items:
                category_items[category] = []
            category_items[category].append(item["itemId"])
    # print(category_items)
    # N-pair samplingをする
    keys = list(category_items.keys())
    # 1. anchorのレイヤを決める
    for positive_category_index, positive_category in enumerate(keys):
        p_items = category_items[positive_category]
        # 2. anchorを決める
        for anchor_index, anchor_item in enumerate(p_items):
            # 3. そのレイヤはすべてpositive
            for positive_item in p_items[anchor_index+1:]:
                # 4. それ以外はすべてnegative
                for negative_category in keys[positive_category_index+1:]:
                    n_items = category_items[negative_category]
                    # 5. negativeのアイテムを決める
                    for negative_item in n_items:
                        # anchorと同じコーディネートに含まれるアイテムはネガティブじゃない
                        data.append([anchor_item, positive_item, negative_item])


【******************* 】

In [47]:
# DataFrameを作成
df = pd.DataFrame(data[1:]).dropna()

# CSVファイルに保存
df.to_csv('C:/Users/yuuta/Documents/fashion/model_learning/versatility/data/triplet_n_pair_train.csv', index=False)

In [48]:
print(len(data))
data

222134435


[[37419896, 8210762, 35578227],
 [37419896, 8210762, 36214783],
 [37419896, 8210762, 37162284],
 [37419896, 8210762, 8173177],
 [37419896, 8210762, 36257678],
 [37419896, 8210762, 35825023],
 [37419896, 8210762, 6773088],
 [37419896, 8210762, 31597304],
 [37419896, 8210762, 35566873],
 [37419896, 8210762, 46433537],
 [37419896, 8210762, 8995055],
 [37419896, 8210762, 42741231],
 [37419896, 8210762, 9420545],
 [37419896, 8210762, 31585895],
 [37419896, 8210762, 16423407],
 [37419896, 8210762, 36786689],
 [37419896, 8210762, 32642224],
 [37419896, 8210762, 20058211],
 [37419896, 8210762, 33490397],
 [37419896, 8210762, 15337798],
 [37419896, 8210762, 42838935],
 [37419896, 8210762, 38959091],
 [37419896, 8210762, 20606841],
 [37419896, 8210762, 15848851],
 [37419896, 8210762, 13954348],
 [37419896, 8210762, 9545596],
 [37419896, 8210762, 10891978],
 [37419896, 8210762, 5551991],
 [37419896, 8210762, 11788067],
 [37419896, 8210762, 41552782],
 [37419896, 8210762, 29421493],
 [37419896, 82

In [49]:
data = pd.read_csv('C:/Users/yuuta/Documents/fashion/model_learning/versatility/data/triplet_n_pair_train.csv', header=None, encoding="utf-8")

In [50]:
df.head()

,0,1,2
0,37419896,8210762,36214783
1,37419896,8210762,37162284
2,37419896,8210762,8173177
3,37419896,8210762,36257678
4,37419896,8210762,35825023
